# Collecting Brazil's 2022 presidential election polls data
> A tutorial on how to collect, filter and prepare for analysis Brazil's 2022 presidential election polls data.

- toc: true 
- badges: true
- comments: true
- categories: [data-science]
- hide: false
- search_exclude: false
- image: images/chart-preview.png

## Context

This year [more than 146 million fellow Brazilian](https://www.tse.jus.br/eleitor/estatisticas-de-eleitorado/consulta-quantitativo) are going to choose our new President...

## Data sources

### Pool data
- [Poder360 polls database](https://www.poder360.com.br/banco-de-dados/)
### Voters statistics

- [TSE: Voters statistics by Region](https://www.tse.jus.br/eleitor/estatisticas-de-eleitorado/consulta-quantitativo)
- [TSE: Voters statistics by sex and age](https://www.tse.jus.br/eleitor/estatisticas-de-eleitorado/estatistica-do-eleitorado-por-sexo-e-faixa-etaria)

### Dashboard
- [App](https://chance-lula-ganhar-1o-turno.github.io/)
- [Repositório: Chance de Lula ganhar no primeiro turno](https://github.com/chance-lula-ganhar-1o-turno)
  

## Analysis

### Importing libraries

In [ ]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import dirichlet
from pathlib import Path
from tqdm.notebook import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load and transform data

In [ ]:
#noshow 
columns_to_use = ["pesquisa_id", "cenario_id", "turno", "partido",
                  "candidato", "cenario_descricao", "instituto",
                  "percentual", "data_pesquisa", "qtd_entrevistas",
                  "qtd_mencoes","grupo"]

HDI = [0.025, 0.5, 0.975] # quantiles

In [ ]:
def enrich_and_filter(df: pd.DataFrame) ->pd.DataFrame:
    df['qtd_mencoes'] = df.percentual * df.qtd_entrevistas / 100.0
    df['grupo'] = 'OUTROS'
    df.loc[(df.partido == 'PT'), 'grupo'] = 'LULA'
    df.loc[(df.partido.isna()), 'grupo'] = 'NULO'

    return df[columns_to_use]

In [ ]:
def get_most_recent_file() -> Path:
    data_dir = Path('../assets/data/elections2022/')
    return sorted(data_dir.glob('2022*.csv'))[-1]
print(get_most_recent_file())

In [ ]:
def get_most_recent_df() -> pd.DataFrame:
    return enrich_and_filter(pd.read_csv(get_most_recent_file(),
                                         sep=',',
                                         parse_dates=True,
                                         dayfirst=True)
                             )


In [ ]:
raw_df = get_most_recent_df()
raw_df.head()

In [ ]:
def prepare_df(raw_df: pd.DataFrame) -> pd.DataFrame:
    table_df = raw_df.pivot_table(values=['qtd_mencoes'],
                        index=['data_pesquisa', 'instituto'],
                        columns=['grupo'],
                        aggfunc=np.sum)
    table2 = table_df.reset_index(col_level=1).copy()
    table2.columns = [a[1] for a in table2.columns.to_flat_index()]
    table2["TOTAL"] = table2['LULA'] + table2['NULO'] + table2['OUTROS']
    table2["mes"] = pd.to_datetime(table2['data_pesquisa']).dt.strftime('%Y_%m')
    return table2

In [ ]:
df = prepare_df(raw_df)
### ENQUANTO PODER DATA NAO ATUALIZA (SUA PRÓPRIA PESQUISA)
### DADOS RETIRADOS DAQUI: https://bit.ly/3Ax2c93
df = df.append(pd.DataFrame({"data_pesquisa": ["2022-01-20"],
                        "mes": ["2022_01"],
                        "LULA":[1260],
                        "NULO":[360],
                        "OUTROS":[1350],
                        "TOTAL": [3000],
                        "instituto":["PoderData"]}))
###
df

In [ ]:
def simulate_prob_freq(concentration: np.array, iters=10_000, N=100_000) -> float:
    priors = stats.dirichlet.rvs(concentration, iters)
    cnt = 0
    for i in range(iters):
        votes = stats.multinomial(n=100_000, p = priors[i]).rvs()[0]
        if votes[0] > votes[2]:
            cnt += 1
    return cnt/iters * 100
simulate_prob_freq(np.array([723, 186, 780]))
#stats.multinomial(n=100_000, p = np.array([0.45, 0.11, 0.44])).rvs()

In [ ]:
def run_prob_weekly(simulate_prob):
    prior = np.array([420,130,450])
    posterior = np.array([0,0,0])
    for obj in df.itertuples():
        like = np.array([obj.LULA, obj.NULO, obj.OUTROS])
        posterior = prior + like * 0.5
        print(obj.data_pesquisa, posterior/posterior.sum(), simulate_prob(posterior))
        prior = 1000 * posterior/posterior.sum()

run_prob_weekly(simulate_prob_freq)

In [ ]:
mes = df.groupby("mes").sum().reset_index()
mes

In [ ]:
def run_prob_monthly(simulate_prob):
    prior = np.array([420,130,450])
    posterior = np.array([0,0,0])
    df_by_mes = df.groupby("mes").sum().reset_index()
    for obj in df_by_mes.itertuples():
        # print(obj)
        like = np.array([obj.LULA, obj.NULO, obj.OUTROS])
        posterior = prior + like * 0.5
        print(obj.mes, posterior, simulate_prob(posterior))
        prior = 2000 * posterior/posterior.sum()
run_prob_monthly(simulate_prob_freq)

In [ ]:
def build_model(prior, observed, rw_alpha_mult=1.0, rw_beta_mult=1.0) -> pm.Model:
    observed = observed.astype("int32")
    with pm.Model() as dirichlet_model:
        rw = pm.Gamma("random_walk",
                      alpha=(rw_alpha_mult * prior),
                      beta=(rw_beta_mult * np.ones(3)), shape=(3,)
                      )
        
        previous_month_prior = pm.Dirichlet(
            "prior", a=rw, shape=(3,),
        )

        pm.Multinomial(
            "current", n=observed.sum(), p=(previous_month_prior), observed=observed, shape=(3,)
        )
    
    return dirichlet_model
model = build_model(np.ones(3), np.ones(3))
pm.model_to_graphviz(model)

In [ ]:

def sampling(model: pm.Model, samples=2000, chains=2):
    with model:
        dirichlet_trace = pm.sample(samples, tune=2000, chains=chains, return_inferencedata=True, progressbar=False)
        ppc = pm.fast_sample_posterior_predictive(dirichlet_trace, 5000)
    return dirichlet_trace , ppc

def calc_prob_lula_win(model, trace, niter=30, ppc=None):
    simul = []
    if ppc:
        lula = ppc['current'][:, 0]
        others = ppc['current'][:, 2]
        simul.append((lula > others).mean() * 100.0)
    for _ in tqdm(range(niter)):
        with model:
            ppc = pm.fast_sample_posterior_predictive(trace, 10000)
        lula = ppc['current'][:, 0]
        others = ppc['current'][:, 2]
        simul.append((lula > others).mean() * 100.0)
    simul = np.array(simul)
    return np.quantile(simul, q=HDI)

### Weekly update

In [ ]:
flat_prior = prior = np.array([400.0, 130.0, 470.0])

df_by_mes = df.groupby("mes").sum().reset_index()
# for obj in df_by_mes.itertuples():

print(df)
for obj in df.itertuples():
    observed = np.array([obj.LULA, obj.NULO, obj.OUTROS])
    print(f">>> observed = {observed}")
    model = build_model(prior, observed, rw_alpha_mult=10.0)
    trace, ppc = sampling(model)
    posterior = ppc['current']
    # print(f">>> posterior.shape={posterior.shape}")
    prior = dirichlet.mle(posterior/posterior.sum(axis=1).reshape(-1, 1))
    # print('dirichlet.mle(posterior/posterior.sum(axis=1).reshape(-1, 1))=,', prior)
    qs = calc_prob_lula_win(model, trace, ppc=ppc)
    # print(f">>> mes={obj.mes}, updaed_prior={prior},  qs={qs}% <<<")
    print(f">>> data_pesquisa={obj.data_pesquisa}, updaed_prior={prior},  qs={qs}% <<<");
    post_prob = posterior/posterior.sum(axis=1).reshape(-1, 1)
    print(f"lula=", np.quantile(post_prob[:, 0], q=HDI))
    print(f"nulo=", np.quantile(post_prob[:, 1], q=HDI))
    print(f"demais=", np.quantile(post_prob[:, 2], q=HDI))
    

In [ ]:
# az.plot_ppc(az.from_pymc3(posterior_predictive=ppc, model=model), figsize=(10,5));
l = [1,2,3]
print(*l)

In [ ]:
az.summary(trace)

In [ ]:
# fig, ax = plt.subplots(2, figsize=(10,6))
lula = ppc['current'][:,0] / ppc['current'].sum(axis=1) * 100
nulo = ppc['current'][:,1] / ppc['current'].sum(axis=1) * 100.00
demais = ppc['current'][:,2] / ppc['current'].sum(axis=1) * 100
az.plot_posterior({'Lula': lula, 'Demais': demais});
#az.plot_posterior({}, ax=ax[1])

In [ ]:
# a0 = np.array([100, 299, 100])
# D0 = np.random.dirichlet(a0, 1000)
# D0
az.plot_posterior({'Lula': lula, 'Demais': demais, 'Nulo/Branco':nulo});
fig, ax = plt.subplots(figsize=(10, 5), dpi=100)
# ax.set_xlim(40, 50)
post_pred = pd.DataFrame({'Lula': lula, 'Nulo': nulo, 'Demais': demais})
sns.kdeplot(data=post_pred, ax=ax, palette=['red','yellow','black'], linewidth=2.5, gridsize=500);

## MONTHLY

In [ ]:
flat_prior = prior = np.array([420.0, 130.0, 450.0])

df_by_mes = df.groupby("mes").sum().reset_index()
for obj in df_by_mes.itertuples():
    observed = np.array([obj.LULA, obj.NULO, obj.OUTROS])
    print(f">>> observed = {observed}")
    model = build_model(prior, observed)
    trace, ppc = sampling(model)
    posterior = ppc['current']
    # print(f">>> posterior.shape={posterior.shape}")
    prior = dirichlet.mle(posterior/posterior.sum(axis=1).reshape(-1, 1))
    # print('dirichlet.mle(posterior/posterior.sum(axis=1).reshape(-1, 1))=,', prior)
    qs = calc_prob_lula_win(model, trace, 100, ppc=ppc)
    print(f">>> mes={obj.mes}, updaed_prior={prior},  qs={[str(q)+'%' for q in qs]} <<<")
    # print(f">>> data_pesquisa={obj.data_pesquisa}, updaed_prior={prior},  qs={qs}% <<<");
    post_prob = posterior/posterior.sum(axis=1).reshape(-1, 1)
    print(f"lula=", np.quantile(post_prob[:, 0], q=HDI))
    print(f"nulo=", np.quantile(post_prob[:, 1], q=HDI))
    print(f"demais=", np.quantile(post_prob[:, 2], q=HDI))



In [ ]:
az.summary(trace)

In [ ]:
lula = ppc['current'][:,0] / ppc['current'].sum(axis=1) * 100.00
nulo = ppc['current'][:,1] / ppc['current'].sum(axis=1) * 100.00
demais = ppc['current'][:,2] / ppc['current'].sum(axis=1) * 100.00
az.plot_posterior({'Lula': lula, 'Demais': demais, 'Nulo/Branco':nulo});

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=100)
# ax.set_xlim(40, 50)
post_pred = pd.DataFrame({'Lula': lula, 'Nulo': nulo, 'Demais': demais})
sns.kdeplot(data=post_pred, ax=ax, palette=['red','yellow','black'], linewidth=2.5, gridsize=500);

In [ ]:
(post_pred.Lula > post_pred.Demais).mean()

## DEZEMBRO

In [ ]:
# NOVEMBER
prior = pd.array([6049, 1508, 7382]).astype("float32")
# DECEMBER
observed = pd.array([7376, 1953, 7578])
                    
with pm.Model() as dirichlet_model:
        
    beta = pm.Beta("beta", alpha=4, beta=1)
    
    november_prior = pm.Dirichlet(
        "november_prior",
        # Lula, BrancosNulosNaoSabem, Demais candidatos
        a=prior * beta,
    )
    
    december_like = pm.Multinomial(
        "december_like", n=observed.sum(), p=november_prior, observed=observed,
    )
    

In [ ]:
with dirichlet_model:
   dirichlet_trace = pm.sample(5000, chains=2, return_inferencedata=False, cores=2, progressbar=False) # 20K samples
   ppc = pm.sample_posterior_predictive(
        dirichlet_trace, random_seed=1777,
   )
   az.plot_trace(dirichlet_trace)

In [ ]:
az.summary(dirichlet_trace)

In [ ]:
print(ppc)
ppc['december_like'].mean(axis=0)

In [ ]:
lula = ppc['december_like'][:, 0]
demais = ppc['december_like'][:, 2]
lula.shape,demais.shape

In [ ]:
(lula > demais).mean()

# Verifica conjugado

In [ ]:
# NOVEMBER
prior_data = pd.array([43, 11, 45]).astype("float32")
# DECEMBER
observed = pd.array([46, 12, 41])
                    
with pm.Model() as m:
        
    prior = pm.Dirichlet("prior", a=prior_data)
    
    pm.Multinomial("like", n=observed.sum(), p=prior, observed=observed)
    trace = pm.sample(5000, chains=2, return_inferencedata=True, cores=2, progressbar=False) # 20K samples

In [ ]:
az.summary(trace)

In [ ]:
obs2 = prior_data + observed
with pm.Model() as m2:
        
    prior = pm.Dirichlet("prior", a=np.array([1.0, 1.0, 1.0]))
    
    pm.Multinomial("like", n=obs2.sum(), p=prior, observed=obs2)
    trace2 = pm.sample(5000, tune=2000, chains=2, return_inferencedata=True, cores=2, progressbar=False) # 20K samples

In [ ]:
az.summary(trace2)

# Janeiro/Dezembro

In [ ]:
import numpy as np

T1 = 2_000 # 
T2 = 1_500 # EXAME/IDEIA
T3 = 1_000 # XP/IPESPE
# Mês atual: Lula, BrancosNulosNaoSabem, Demais candidatos
current_observed = np.array([1955, 535, 2010])

print(current_observed.sum())

december_data = np.array([7356, 1973, 7578]).astype("float32")

with pm.Model() as dirichlet_model:
    
    beta = pm.Beta("beta", alpha=6, beta=1)
    
    december_prior = pm.Dirichlet(
        "prior",
        # Mes anterior: Lula, BrancosNulosNaoSabem, Demais candidatos
        a = december_data * beta,
        # a = [1.0,1.0,1.0],
        shape=(3,),
    )
    
    january_like = pm.Multinomial(
        "like", n=current_observed.sum(), p=december_prior, observed=current_observed, shape=(3,)
    )
pm.model_to_graphviz(dirichlet_model)    

In [ ]:
with dirichlet_model:
   dirichlet_trace = pm.sample(5000, chains=4, return_inferencedata=True, cores=4, progressbar=False) # 20K samples
   ppc = pm.sample_posterior_predictive(
        dirichlet_trace, random_seed=1777
   )
   az.plot_trace(dirichlet_trace, figsize=(22,8))
   dirichlet_trace.extend(az.from_dict(posterior_predictive=ppc))

In [ ]:
az.summary(dirichlet_trace)

In [ ]:
az.plot_posterior(dirichlet_trace);

In [ ]:
az.plot_energy(dirichlet_trace)

In [ ]:
lula = ppc['like'][:, 0]
demais = ppc['like'][:, 2]
print(lula.shape,demais.shape)
prob_lula = (lula > demais).mean() * 100.0
print(f"Probabilidade de Lula vencer no primeiro turno é de: {prob_lula:.2f}%", )

In [ ]:
# create figure and axes
fig, ax = plt.subplots(figsize=(10, 5), dpi=100)
ax.set_xlim(1800, 2200)
sns.kdeplot(data=post_pred, ax=ax, palette=['red','yellow','black'], linewidth=2.5, gridsize=500);

## Playground

In [ ]:
import math
1 - stats.norm.cdf(0, loc=-1, scale=math.sqrt(5))

In [ ]:
import scipy as sp
true_p = sp.stats.dirichlet(6.0 * np.array([0.45, 0.3, 0.15, 0.9, 0.01])).rvs(size=10)
observed = np.vstack([sp.stats.multinomial(n=50, p=p_i).rvs() for p_i in true_p])



In [ ]:
observed.sum(axis=1)

In [ ]:
current_observed.sum(axis=0)

In [ ]:
current_observed.sum(axis=1)

In [ ]:
current_observed

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
from datetime import datetime
import plotly.io as pio
# Default is plotly_mimetype+notebook, but jekyll fails to parse plotly_mimetype.
pio.renderers.default = 'notebook_connected'

# Inject the missing require.js dependency.
from IPython.display import display, HTML
js = '<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js" integrity="sha512-c3Nl8+7g4LMSTdrm621y7kf9v3SDPnhxLNhcjFJbKECVnmZHTdo+IRO05sNLTH/D3vA6u1X32ehoLC7WFVdheg==" crossorigin="anonymous"></script>'
display(HTML(js))
import plotly.express as px

df = px.data.gapminder()
px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])


In [ ]:
# hide_input
# This cell is required for the export to HTML to work.


In [ ]:
observed_prior = np.array([7356., 1973., 7578.])
prior = []
N = 4000
for it in range(N):
    b = stats.beta.rvs(a=5, b=1, size=3)
    # print(b)
    # print(observed_prior * b)
    diric = stats.dirichlet.rvs(observed_prior * b,)
    # print(diric[0])
    prior.append(diric[0])
prior = np.array(prior)
print(prior[:4])

In [ ]:

print((prior[:,0] > prior[:,2]).mean())
print(prior.shape)


In [ ]:
posterior = []
for it in range(N):
    mn = stats.multinomial.rvs(n=3000, p=[1340/3000, 370/3000, 1290/3000])
    posterior.append(mn * prior[it])
posterior = np.array(posterior)



In [ ]:
post = posterior#/posterior.sum(axis=1).reshape(4000,1)
print(post.sum(axis=1))
print(post[:10, 0])
print(post[:10, 2])

In [ ]:
lula = post[:, 0]
others = post[:, 2]
print((lula > others).mean())

In [ ]:
simul = []
for it in range(N):
    idx = np.random.choice(N,1)[0]
    #print(idx)
    #print(post[idx])
    dir2 = stats.dirichlet.rvs(post[idx])[0]
    simul.append(dir2)
dir2 = np.array(simul)

In [ ]:
dir2

In [ ]:
lula = dir2[:, 0]
others = dir2[:, 2]
print((lula > others).mean())

In [ ]:
np.arange(10)

In [ ]:
np.linspace(0,1, 101)

In [ ]:
x = np.arange(5,10)

In [ ]:
np.concatenate([[x[0] - 5], x, [x[-1] + 5]])

In [ ]:
1.96 * np.sqrt((0.5**2)/3000)

In [ ]:
1./(1./0.01789 + 1./0.0200)

In [ ]:
1/100

In [ ]:
1/(1./0.01789 + 1./0.0200)